In [1]:
import re
import pickle
import string
import copy
from pathlib import Path

import pandas as pd
import numpy as np
import spacy
import requests
import feedparser
from bs4 import BeautifulSoup

In [2]:
path = 'https://www.cisa.gov/uscert/ncas/alerts.xml'
data_path = Path("../data")

In [3]:
d = feedparser.parse(path)
d['feed']['title']

'CISA Alerts'

In [4]:
d.feed.title

'CISA Alerts'

In [5]:
d.feed.link

'https://us-cert.cisa.gov/'

In [6]:
d.feed.description

'Alerts warn about vulnerabilities, incidents, and other security issues that pose a significant risk.'

In [7]:
d.entries[0].title

'AA22-321A: #StopRansomware: Hive Ransomware'

In [8]:
d.entries[0].link

'https://us-cert.cisa.gov/ncas/alerts/aa22-321a'

In [9]:
d.entries[0].description[:100]

'Original release date: November 17, 2022 | Last revised: November 25, 2022<br /><h3>Summary</h3><p c'

In [10]:
d.entries[0].published

'Thu, 17 Nov 2022 17:00:00 +0000'

In [11]:
d.entries[0].published_parsed

time.struct_time(tm_year=2022, tm_mon=11, tm_mday=17, tm_hour=17, tm_min=0, tm_sec=0, tm_wday=3, tm_yday=321, tm_isdst=0)

In [12]:
d.entries[0].id

'18160 at https://us-cert.cisa.gov'

# get alerts beautifulsoup

In [13]:
import requests
from bs4 import BeautifulSoup

In [14]:
url='https://www.cisa.gov/uscert/ncas/alerts'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

In [15]:
soup.title

<title>Alerts | CISA</title>

In [16]:
alert_url='https://www.cisa.gov/uscert/ncas/alerts/aa22-321a'
alert_page = requests.get(alert_url)
alert_soup = BeautifulSoup(alert_page.text, 'html.parser')
soup.title

<title>Alerts | CISA</title>

In [17]:
%%time
#get content for each alert on the first 10 pages of alerts
alerts_new = []
soup_alerts_path = Path('soup_alerts.pkl')
if soup_alerts_path.exists():
    with open(soup_alerts_path, "rb") as sap:
        alerts = pickle.load(sap)
else:
    # I think there's only 8 or so pages
    for p_num in range(10): #look at 10 pages of results
        #get the the content from this page
        url='https://www.cisa.gov/uscert/ncas/alerts?page=' + str(p_num)
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        table = soup.find_all(class_="views-field views-field-title")

        for i in range(len(table)):
            #get the alert details from this line in the list
            link = table[i].find('a')['href']
            alert_id = table[i].find(class_="field-content").contents[0]
            title = table[i].find('a').text

            #get alert content
            alert_url='https://www.cisa.gov/uscert' + link
            alert_page = requests.get(alert_url)
            alert_soup = BeautifulSoup(alert_page.text, 'html.parser')
            alert_date = alert_soup.find(id="ncas-header").find(class_="submitted meta-text").text
            alert_text = alert_soup.find(id="ncas-content").text

            #write the content
            alerts_new.append([link, alert_id, title, alert_date, alert_text])

    with open(soup_alerts_path, "wb") as sap:
        pickle.dump(alerts_new, sap)
    alerts = alerts_new
alerts[0]

CPU times: user 40.7 ms, sys: 6.92 ms, total: 47.6 ms
Wall time: 46.7 ms


['/ncas/alerts/aa22-321a',
 'AA22-321A : ',
 '#StopRansomware: Hive Ransomware',
 '\n                                Original release date: November 17, 2022  | Last revised: November 25, 2022\n                                            ',
 '\n\nSummary\nActions to Take Today to Mitigate Cyber Threats from Ransomware:\n• Prioritize remediating known exploited vulnerabilities.\n• Enable and enforce multifactor authentication with strong passwords\n• Close unused ports and remove any application not deemed necessary for day-to-day operations.\nNote: This joint Cybersecurity Advisory (CSA) is part of an ongoing #StopRansomware effort to publish advisories for network defenders that detail various ransomware variants and ransomware threat actors. These #StopRansomware advisories include recently and historically observed tactics, techniques, and procedures (TTPs) and indicators of compromise (IOCs) to help organizations protect against ransomware. Visit stopransomware.gov to see all #Stop

In [18]:
%%time
headers = ['link', 'alert_id', 'title', 'date', 'text']
df = pd.DataFrame(alerts, columns=headers)
df.head()

CPU times: user 562 µs, sys: 184 µs, total: 746 µs
Wall time: 735 µs


,link,alert_id,title,date,text
0,/ncas/alerts/aa22-321a,AA22-321A :,#StopRansomware: Hive Ransomware,\n Original rel...,\n\nSummary\nActions to Take Today to Mitigate...
1,/ncas/alerts/aa22-320a,AA22-320A :,Iranian Government-Sponsored APT Actors Compro...,\n Original rel...,\n\nSummary\nFrom mid-June through mid-July 20...
2,/ncas/alerts/aa22-294a,AA22-294A :,#StopRansomware: Daixin Team,\n Original rel...,\n\nSummary\nActions to take today to mitigate...
3,/ncas/alerts/aa22-279a,AA22-279A :,Top CVEs Actively Exploited By People’s Republ...,\n ...,\n\nSummary\nThis joint Cybersecurity Advisory...
4,/ncas/alerts/aa22-277a,AA22-277A :,Impacket and Exfiltration Tool Used to Steal S...,\n Original rel...,\n\nSummary\nActions to Help Protect Against A...


# parse alert content

In [19]:
re_cve = re.compile(r'(CVE-\d{4}-\d+)')
re_cve.findall(alerts[0][4])

['CVE-2020-12812',
 'CVE-2021-31207',
 'CVE-2021-34473',
 'CVE-2021-34523',
 'CVE-2021-34473',
 'CVE-2021-34523',
 'CVE-2021-31207',
 'CVE-2021-42321']

In [20]:
re_ttp = re.compile(r'(\[|\s)(T\d{4})\.?\d*(\]|\s)')
[ttp[1] for ttp in re_ttp.findall(alerts[1][4])]

['T1190',
 'T1059',
 'T1562',
 'T1105',
 'T1070',
 'T1136',
 'T1016',
 'T1053',
 'T1021',
 'T1078',
 'T1136',
 'T1090',
 'T1018',
 'T1098',
 'T1003',
 'T1190',
 'T1059',
 'T1098',
 'T1136',
 'T1136',
 'T1053',
 'T1078',
 'T1562',
 'T1070',
 'T1003',
 'T1555',
 'T1018',
 'T1016',
 'T1021',
 'T1090',
 'T1105']

In [21]:
#regex for cve match
re_cve = re.compile(r'(CVE-\d{4}-\d*)')
cves = []
for i in range(len(alerts)):
    alert_cves = sorted(re_cve.findall(alerts[i][4]))
    cves.append(alert_cves)
cves = [set(cve) for cve in cves]

ttps = []
for i in range(len(alerts)):
    alert_ttps = [ttp[1] for ttp in re_ttp.findall(alerts[i][4])]
    alert_ttps = sorted(alert_ttps)
    ttps.append(alert_ttps)
ttps = [set(ttp) for ttp in ttps]

In [22]:
#add list of CVE & TTP to df
df['CVE_list'] = cves
df['TTP_list'] = ttps
print(df.describe())
df.head()

                          link      alert_id  \
count                      286           286   
unique                     286           286   
top     /ncas/alerts/aa22-321a  AA22-321A :    
freq                         1             1   

                                                 title  \
count                                              286   
unique                                             210   
top     Microsoft Updates for Multiple Vulnerabilities   
freq                                                56   

                                                     date  \
count                                                 286   
unique                                                277   
top     \n                                Original rel...   
freq                                                    2   

                                                     text CVE_list TTP_list  
count                                                 286      286      286  
unique

,link,alert_id,title,date,text,CVE_list,TTP_list
0,/ncas/alerts/aa22-321a,AA22-321A :,#StopRansomware: Hive Ransomware,\n Original rel...,\n\nSummary\nActions to Take Today to Mitigate...,"{CVE-2021-31207, CVE-2021-34523, CVE-2021-4232...","{T1112, T1537, T1566, T1059, T1070, T1562, T14..."
1,/ncas/alerts/aa22-320a,AA22-320A :,Iranian Government-Sponsored APT Actors Compro...,\n Original rel...,\n\nSummary\nFrom mid-June through mid-July 20...,{CVE-2021-44228},"{T1136, T1090, T1016, T1059, T1070, T1562, T10..."
2,/ncas/alerts/aa22-294a,AA22-294A :,#StopRansomware: Daixin Team,\n Original rel...,\n\nSummary\nActions to take today to mitigate...,{},"{T1563, T1567, T1078, T1598, T1486, T1190, T10..."
3,/ncas/alerts/aa22-279a,AA22-279A :,Top CVEs Actively Exploited By People’s Republ...,\n ...,\n\nSummary\nThis joint Cybersecurity Advisory...,"{CVE-2021-26855, CVE-2021-42237, CVE-2021-2220...",{}
4,/ncas/alerts/aa22-277a,AA22-277A :,Impacket and Exfiltration Tool Used to Steal S...,\n Original rel...,\n\nSummary\nActions to Help Protect Against A...,"{CVE-2021-27065, CVE-2021-26855, CVE-2021-2685...","{T1090, T1574, T1047, T1083, T1021, T1543, T10..."


In [48]:
df

,link,alert_id,title,date,text,CVE_list,TTP_list,text2,full_path
0,/ncas/alerts/aa22-321a,AA22-321A,#StopRansomware: Hive Ransomware,2022-11-17,\n\nSummary\nActions to Take Today to Mitigate...,"{CVE-2021-31207, CVE-2021-34523, CVE-2021-4232...","{T1112, T1537, T1566, T1059, T1070, T1562, T14...",Actions to Take Today to Mitigate Cyber Threat...,https://www.cisa.gov/ncas/alerts/aa22-321a
1,/ncas/alerts/aa22-320a,AA22-320A,Iranian Government-Sponsored APT Actors Compro...,2022-11-16,\n\nSummary\nFrom mid-June through mid-July 20...,{CVE-2021-44228},"{T1136, T1090, T1016, T1059, T1070, T1562, T10...","From mid-June through mid-July 2022, CISA cond...",https://www.cisa.gov/ncas/alerts/aa22-320a
2,/ncas/alerts/aa22-294a,AA22-294A,#StopRansomware: Daixin Team,2022-10-21,\n\nSummary\nActions to take today to mitigate...,{},"{T1563, T1567, T1078, T1598, T1486, T1190, T10...",Actions to take today to mitigate cyber threat...,https://www.cisa.gov/ncas/alerts/aa22-294a
3,/ncas/alerts/aa22-279a,AA22-279A,Top CVEs Actively Exploited By People’s Republ...,2022-10-06,\n\nSummary\nThis joint Cybersecurity Advisory...,"{CVE-2021-26855, CVE-2021-42237, CVE-2021-2220...",{},This joint Cybersecurity Advisory (CSA) provid...,https://www.cisa.gov/ncas/alerts/aa22-279a
4,/ncas/alerts/aa22-277a,AA22-277A,Impacket and Exfiltration Tool Used to Steal S...,2022-10-04,\n\nSummary\nActions to Help Protect Against A...,"{CVE-2021-27065, CVE-2021-26855, CVE-2021-2685...","{T1090, T1574, T1047, T1083, T1021, T1543, T10...",Actions to Help Protect Against APT Cyber Acti...,https://www.cisa.gov/ncas/alerts/aa22-277a
...,...,...,...,...,...,...,...,...,...
281,/ncas/alerts/TA08-352A,TA08-352A,Microsoft Internet Explorer Data Binding Vulne...,2008-12-17,\n\nSystems Affected\n\nOverview\nMicrosoft In...,{},{},Systems Affected Overview Microsoft Intern...,https://www.cisa.gov/ncas/alerts/TA08-352A
282,/ncas/alerts/TA08-350A,TA08-350A,Apple Updates for Multiple Vulnerabilities,2008-12-15,\n\nSystems Affected\nSystems running Mac OS X...,{},{},Systems Affected Systems running Mac OS X and...,https://www.cisa.gov/ncas/alerts/TA08-350A
283,/ncas/alerts/TA08-344A,TA08-344A,Microsoft Updates for Multiple Vulnerabilities,2008-12-09,\n\nSystems Affected\nMicrosoft Windows and re...,{},{},Systems Affected Microsoft Windows and relate...,https://www.cisa.gov/ncas/alerts/TA08-344A
284,/ncas/alerts/TA08-340A,TA08-340A,Sun Java Updates for Multiple Vulnerabilities,2008-12-05,\n\nSystems Affected\n \n\nOverview\nSun...,{},{},Systems Affected Overview Sun has releas...,https://www.cisa.gov/ncas/alerts/TA08-340A


# NER

In [76]:
# Testing version pattern regex
version_pattern = re.compile(r"(v?[0-9]{1,2})\.[0-9]+\.?[0-9]*")
df[df['text'].apply(lambda x: version_pattern.search(x)) != None].sample(3)['text']

205    \n\nSystems Affected\nMicrosoft WindowsMicroso...
271    \n\nSystems Affected\nOracle Database 11g, ver...
192    \n\nSystems Affected\nMicrosoft Internet Explo...
Name: text, dtype: object

In [74]:
# Some cleaning up of text field before NER
text = copy.deepcopy(df['text'])

def replace_non_printable(raw_text):
    filtered_text = list(filter(lambda x: x in string.printable, raw_text))
    return "".join(filtered_text)

def replace_strings(text, old_text, new_text):
    if isinstance(str_pattern,str):
        new_text = text.str.replace(old_text,new_text)
    else:
        new_text = text.str.replace(old_text,new_text,regex=True)
    return new_text.str.strip()

# List of string patterns to replace. Set as tuple to specify custom replacement string
strings_to_replace= ["Summary", ("\n"," "*2),("\t"," "*2), re.compile(r"DISCLAIMER.*"), re.compile(r"INFORMATION REQUESTED.*")]

for str_pattern in strings_to_replace:
    new_text = " "
    old_text = str_pattern
    if type(old_text) is tuple:
        new_text = str_pattern[1]
        old_text = str_pattern[0]
    text = replace_strings(text, old_text, new_text)


# Remove Nonprintable, CVE IDs, and TTPs
text = text.apply(lambda raw_x: replace_non_printable(raw_x))
text = text.str.replace(re_cve,"",regex=True)
text = text.str.replace(re_ttp,"",regex=True)

# Remove Minor Version Numbers (Assumption here is that we're only tracking larger trends in a product overall not specific version updates)
version_pattern = re.compile(r"(v?[0-9]{1,2})\.[0-9]+\.?[0-9]*(^[0-9]*)")
text = text.apply(lambda x: version_pattern.sub(r"\1", x))

df['text2'] = text
text[0]

'Actions to Take Today to Mitigate Cyber Threats from Ransomware:   Prioritize remediating known exploited vulnerabilities.   Enable and enforce multifactor authentication with strong passwords   Close unused ports and remove any application not deemed necessary for day-to-day operations.  Note: This joint Cybersecurity Advisory (CSA) is part of an ongoing #StopRansomware effort to publish advisories for network defenders that detail various ransomware variants and ransomware threat actors. These #StopRansomware advisories include recently and historically observed tactics, techniques, and procedures (TTPs) and indicators of compromise (IOCs) to help organizations protect against ransomware. Visit stopransomware.gov to see all #StopRansomware advisories and to learn more about other ransomware threats and no-cost resources.  The Federal Bureau of Investigation (FBI), the Cybersecurity and Infrastructure Security Agency (CISA), and the Department of Health and Human Services (HHS) are r

In [24]:
# Clean alert id
df['alert_id'] = df['alert_id'].str.replace(' :','').str.strip()

# Extract date from date field
df['date'] = pd.to_datetime(df['date'].str.replace("Original release date: ","",regex=False).str.replace("\|.*","",regex=True).str.strip())

#convert link to full path
df['full_path'] = 'https://www.cisa.gov' + df['link']

In [25]:
nlp = spacy.load("en_core_web_trf")

In [40]:
ner_list = [pd.Series(this_ner).drop_duplicates() for this_ner in ner]
ner_list

[0                              [stopransomware.gov, ORG]
 1             [The Federal Bureau of Investigation, ORG]
 2                                             [FBI, ORG]
 3      [the Cybersecurity and Infrastructure Security...
 4                                            [CISA, ORG]
 5      [the Department of Health and Human Services, ...
 6                                             [HHS, ORG]
 13                [MITRE ATT&CK for Enterprise, PRODUCT]
 14                                       [Hive, PRODUCT]
 18                                 [Communications, ORG]
 19                   [Healthcare and Public Health, ORG]
 21                    [Remote Desktop Protocol, PRODUCT]
 23                                    [FortiOS, PRODUCT]
 24                             [Microsoft Exchange, ORG]
 25                      [Microsoft Exchange Server, ORG]
 27                                        [Windows, ORG]
 29                           [Windows Defender, PRODUCT]
 31           

In [28]:
%%time
# Extract entity from each text2
ner_pickle_path = data_path / "ner_text2.pkl"
if 'ner2' in df.columns:
    print("Not running the entity recognition, since the data already exists as 'ner' in the dataframe.")
elif ner_pickle_path.exists():
    with open(ner_pickle_path, "rb") as npp:
        df["ner2"] = pickle.load(ner_pickle_path)
else:
    ner2 = []
    for i in range(len(df['text2'])):
        #nlp this doc
        this_doc = nlp(df['text2'][i])

        #select get entities from this doc
        this_ner = []
        for ent in this_doc.ents:
            #choose from list of 
            ent_labels = ['ORG','PRODUCT','GPE','PERSON','NORP']
            if ent.label_ in ent_labels:
                this_ner.append([ent.text, ent.label_])

        #append ner for this doc
        ner2.append(this_ner)
    with open(ner_pickle_path, "wb") as npp:
        pickle.dump(ner2, npp)
    df['ner2'] = ner2
df['ner2']

TypeError: file must have 'read' and 'readline' attributes

In [27]:
%%time
#extract entity from each text
ner_pickle_path = Path("alerts") / "ner.pkl"
if 'ner' in df.columns:
    print("Not running the entity recognition, since the data already exists as 'ner' in the dataframe.")
elif ner_pickle_path.exists():
    with open(ner_pickle_path, "rb") as npp:
        df['ner'] = pickle.load(ner_pickle_path)
else:
    ner = []
    for i in range(len(alerts)):
        #nlp this doc
        this_doc = nlp(alerts[i][4])

        #select get entities from this doc
        this_ner = []
        for ent in this_doc.ents:
            #choose from list of 
            ent_labels = ['ORG','PRODUCT','GPE','PERSON','NORP']
            if ent.label_ in ent_labels:
                this_ner.append([ent.text, ent.label_])

        #append ner for this doc
        ner.append(this_ner)
    with open(ner_pickle_path, "wb") as npp:
        pickle.dump(ner, npp)

    # add ner into df. leave duplicates for other opportunity
    df['ner'] = ner

df['ner']

Token indices sequence length is longer than the specified maximum sequence length for this model (779 > 512). Running this sequence through the model will result in indexing errors


FileNotFoundError: [Errno 2] No such file or directory: 'alerts/ner.pkl'

In [28]:
%%time
# Extract entity from each text2
ner_pickle_path = data_path / "ner_text2.pkl"
if 'ner2' in df.columns:
    print("Not running the entity recognition, since the data already exists as 'ner' in the dataframe.")
elif ner_pickle_path.exists():
    with open(ner_pickle_path, "rb") as npp:
        df["ner2"] = pickle.load(ner_pickle_path)
else:
    ner2 = []
    for i in range(len(df['text2'])):
        #nlp this doc
        this_doc = nlp(df['text2'][i])

        #select get entities from this doc
        this_ner = []
        for ent in this_doc.ents:
            #choose from list of 
            ent_labels = ['ORG','PRODUCT','GPE','PERSON','NORP']
            if ent.label_ in ent_labels:
                this_ner.append([ent.text, ent.label_])

        #append ner for this doc
        ner2.append(this_ner)
    with open(ner_pickle_path, "wb") as npp:
        pickle.dump(ner2, npp)
    df['ner2'] = ner2
df['ner2']

TypeError: file must have 'read' and 'readline' attributes

In [29]:
# Only write pickle if df has values
if df is not None and not df.empty:
    df.to_pickle("./alerts_df.pkl")

In [30]:
# Load in the dataframe if it's not there already
if "df" not in locals():
    df = pd.read_pickle("./alerts_df.pkl")

In [31]:
df.head()

,link,alert_id,title,date,text,CVE_list,TTP_list,text2,full_path
0,/ncas/alerts/aa22-321a,AA22-321A,#StopRansomware: Hive Ransomware,2022-11-17,\n\nSummary\nActions to Take Today to Mitigate...,"{CVE-2021-31207, CVE-2021-34523, CVE-2021-4232...","{T1112, T1537, T1566, T1059, T1070, T1562, T14...",Actions to Take Today to Mitigate Cyber Threat...,https://www.cisa.gov/ncas/alerts/aa22-321a
1,/ncas/alerts/aa22-320a,AA22-320A,Iranian Government-Sponsored APT Actors Compro...,2022-11-16,\n\nSummary\nFrom mid-June through mid-July 20...,{CVE-2021-44228},"{T1136, T1090, T1016, T1059, T1070, T1562, T10...","From mid-June through mid-July 2022, CISA cond...",https://www.cisa.gov/ncas/alerts/aa22-320a
2,/ncas/alerts/aa22-294a,AA22-294A,#StopRansomware: Daixin Team,2022-10-21,\n\nSummary\nActions to take today to mitigate...,{},"{T1563, T1567, T1078, T1598, T1486, T1190, T10...",Actions to take today to mitigate cyber threat...,https://www.cisa.gov/ncas/alerts/aa22-294a
3,/ncas/alerts/aa22-279a,AA22-279A,Top CVEs Actively Exploited By People’s Republ...,2022-10-06,\n\nSummary\nThis joint Cybersecurity Advisory...,"{CVE-2021-26855, CVE-2021-42237, CVE-2021-2220...",{},This joint Cybersecurity Advisory (CSA) provid...,https://www.cisa.gov/ncas/alerts/aa22-279a
4,/ncas/alerts/aa22-277a,AA22-277A,Impacket and Exfiltration Tool Used to Steal S...,2022-10-04,\n\nSummary\nActions to Help Protect Against A...,"{CVE-2021-27065, CVE-2021-26855, CVE-2021-2685...","{T1090, T1574, T1047, T1083, T1021, T1543, T10...",Actions to Help Protect Against APT Cyber Acti...,https://www.cisa.gov/ncas/alerts/aa22-277a


# alert to CVE edges

In [32]:
#alert to cve edges
alert_cve = df[['alert_id','CVE_list']].explode('CVE_list')
alert_cve = alert_cve.drop_duplicates().dropna().reset_index(drop=True)
alert_cve.to_csv(data_path / 'alert_cve_edge.csv', index=False)
alert_cve

,alert_id,CVE_list
0,AA22-321A,CVE-2021-31207
1,AA22-321A,CVE-2021-34523
2,AA22-321A,CVE-2021-42321
3,AA22-321A,CVE-2020-12812
4,AA22-321A,CVE-2021-34473
...,...,...
391,TA09-195A,CVE-2008-0015
392,TA09-195A,CVE-2009-1537
393,TA09-133B,CVE-2009-1493
394,TA09-133B,CVE-2009-1492


# alert nodes

In [33]:
alerts_nodes = df[['alert_id','full_path','title','date']].drop_duplicates()
alerts_nodes.to_csv(data_path / 'alert_nodes.csv')
alerts_nodes

,alert_id,full_path,title,date
0,AA22-321A,https://www.cisa.gov/ncas/alerts/aa22-321a,#StopRansomware: Hive Ransomware,2022-11-17
1,AA22-320A,https://www.cisa.gov/ncas/alerts/aa22-320a,Iranian Government-Sponsored APT Actors Compro...,2022-11-16
2,AA22-294A,https://www.cisa.gov/ncas/alerts/aa22-294a,#StopRansomware: Daixin Team,2022-10-21
3,AA22-279A,https://www.cisa.gov/ncas/alerts/aa22-279a,Top CVEs Actively Exploited By People’s Republ...,2022-10-06
4,AA22-277A,https://www.cisa.gov/ncas/alerts/aa22-277a,Impacket and Exfiltration Tool Used to Steal S...,2022-10-04
...,...,...,...,...
281,TA08-352A,https://www.cisa.gov/ncas/alerts/TA08-352A,Microsoft Internet Explorer Data Binding Vulne...,2008-12-17
282,TA08-350A,https://www.cisa.gov/ncas/alerts/TA08-350A,Apple Updates for Multiple Vulnerabilities,2008-12-15
283,TA08-344A,https://www.cisa.gov/ncas/alerts/TA08-344A,Microsoft Updates for Multiple Vulnerabilities,2008-12-09
284,TA08-340A,https://www.cisa.gov/ncas/alerts/TA08-340A,Sun Java Updates for Multiple Vulnerabilities,2008-12-05


# dedupe

In [34]:
import os
import csv
import re
import logging
import optparse

import dedupe
from unidecode import unidecode

In [35]:
#alert to entity edges
df_ner = df[['alert_id','ner2']].explode('ner2')
df_ner[['label','type']] = pd.DataFrame(df_ner.ner2.tolist(), index= df_ner.index)
df_ner=df_ner.reset_index(level=0)
df_ner=df_ner.drop(columns='ner2')
df_ner

KeyError: "['ner2'] not in index"

In [ ]:
ner_dedupe = df_ner[['label', 'type']]
ner_dedupe = ner_dedupe.drop_duplicates()
ner_dedupe = ner_dedupe.reset_index(drop=True)
ner_dedupe.to_csv(data_path / 'alert_ner.csv', index=True, index_label = 'Id', columns=['label','type'])
ner_dedupe

In [ ]:
#Do a little bit of data cleaning with the help of Unidecode and Regex. Things like casing, extra spaces, quotes and new lines can be ignored.

def preProcess(column):
#
    column = unidecode(column)
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
#If data is missing, indicate that by setting the value to None

    if not column:
        column = None
    return column
#Read in our data from a CSV file and create a dictionary of records, where the key is a unique record ID and each value is dict

def readData(filename):
#
    data_d = {}
    with open(filename, encoding="utf8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
            row_id = int(row['Id'])
            data_d[row_id] = dict(clean_row)

    return data_d


In [ ]:
input_file = data_path / 'alert_ner.csv'
output_file = data_path / 'ner_cluster.csv'
training_file = data_path / 'ner_training.json'

In [ ]:
data_d=readData(input_file)

In [ ]:
#define dedupe
fields = [
            {'field': 'label', 'type': 'String'},            {'field': 'type', 'type': 'String'}
            ]
deduper = dedupe.Dedupe(fields)

In [ ]:
print('preparing training...')
if os.path.exists(training_file):
    print('reading labeled examples from ', training_file)
    with open(training_file, 'rb') as f:
        deduper.prepare_training(data_d, f)
else:
    deduper.prepare_training(data_d)

In [ ]:
# print('starting active labeling...')
# dedupe.console_label(deduper)

In [ ]:
deduper.train()

with open(training_file, 'w') as tf:
        deduper.write_training(tf)

In [ ]:
print('clustering...')
clustered_dupes = deduper.partition(data_d, 0.5)

print('# duplicate sets', len(clustered_dupes))

In [ ]:
cluster_membership = {}
for cluster_id, (records, scores) in enumerate(clustered_dupes):
    for record_id, score in zip(records, scores):
        cluster_membership[record_id] = {
            "Cluster ID": cluster_id,
            "confidence_score": score
        }

with open(output_file, 'w') as f_output, open(input_file, encoding="utf8") as f_input:

    reader = csv.DictReader(f_input)
    fieldnames = ['Cluster ID', 'confidence_score'] + reader.fieldnames

    writer = csv.DictWriter(f_output, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        row_id = int(row['Id'])
        row.update(cluster_membership[row_id])
        writer.writerow(row)

# alert to ner edge

In [ ]:
df_clusters = pd.read_csv(data_path / 'ner_cluster.csv')

** TODO **
1. Remove punctuation from labels
2. Remove any labels that have only < 2 characters once punctuation is removed
3. Plot the confidence scores and check accuracy around the chosen threshold
4. Figure out why so many matches are not correct. "stopransomware" matching with CISA, FBI doesn't make sense, same with "MS Windows and Firefox, Thunderbird, SeaMonkey.
**5. Match "PRODUCT" type entities to CPE list**



In [ ]:
df_ner_clusters = df_ner.merge(df_clusters, left_on='index', right_on='Id', suffixes=["_x","_y"]).drop(columns="index")
df_ner_clusters

In [ ]:
alert_ner_edge = pd.DataFrame(df_ner_clusters.groupby(by=['alert_id','label_x']).size())
alert_ner_edge = alert_ner_edge.reset_index()
alert_ner_edge.to_csv(data_path / 'alert_ner_edge.csv', index=False, header=['alert_id','ner','weight'])
alert_ner_edge

# ner node

In [ ]:
ner_node = df_ner_clusters.groupby(by=['label_x','Cluster ID']).size()
ner_node = ner_node.reset_index()
ner_node.to_csv(data_path / 'ner_node.csv', index=False, header=['ner','cluster id','weight'])
ner_node